In [1]:
import os

import numpy as np
import pandas as pd
import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 40)

%matplotlib inline

In [11]:
data_directory = '../../../Data/2_eye/Clean/'

In [45]:
class Tidy(object):
    def __init__(self, initials, sub_no=0):
        self.directory = data_directory
        self.subject = initials
        self.sub_no = sub_no
        
        self.tcols = [
            'tstamp', 'eyex', 'eyey', 'dt', 'tile'
        ]
        
        self.D = self.load()
        self.T = self.make_tidy()
        
    def load(self):
        eye = pd.read_csv(
            '{}new_{}_events.csv'.format(self.directory, self.subject)
        )
        
        eye = eye.loc[~pd.isnull(eye.exp_time), :]
        eye.exp_time = (eye.exp_time*1000).astype(int)
        print(eye.columns)
        
        return eye
    
    def make_tidy(self):
        ts = self.D.exp_time.astype(int)
        tidy = pd.DataFrame(
            index=np.arange(ts.max() + 1 - ts.min()), 
            columns=self.tcols
        )
        tidy.loc[:, 'tstamp'] = np.arange(ts.min(), ts.max()+1, 1)
        
        tidy.loc[
            tidy.tstamp.isin((1000*self.D.end.values).astype(int)), 
            ['eyex', 'eyey', 'dt', 'tile']
        ] = self.D.loc[:, ['raw_xloc', 'raw_yloc', 'dur', 'tile']].values
        
        tidy = tidy.loc[pd.notnull(tidy.eyex), :]
        tidy.dt = (tidy.tstamp - tidy.tstamp.shift(1)).shift(-1)
        return tidy
    
    

In [46]:
inits = [f[:2] for f in os.listdir('../../../Data/2_eye/Game/Raw/') if f[-3:]=='csv']
t = Tidy(inits[0])

Index(['etstart', 'etend', 'dur', 'raw_xloc', 'raw_yloc', 'etype', 'start',
       'end', 'res', 'subject', 'gi', 'mi', 'status', 'exp_time',
       'exp_move_start', 'bp', 'wp', 'transx', 'transy', 'tile'],
      dtype='object')
